## Model Loading

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

C:\Users\Windows 10 PRO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_response(model, tokenizer, prompt, max_length=250):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Cria a máscara de atenção e o pad token id -> Sla o que isso quer dizer na vdd
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [3]:
model_path = "./RAW_MODEL/" # Diretório onde o modelo está
# Carrega o modelo e o tokenizador
my_chat_model = GPT2LMHeadModel.from_pretrained(model_path)
my_chat_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

## Aux functions

In [4]:
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
from pygame import mixer
import time

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# Função para reconhecimento de voz
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Ouvindo...\n")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio, language="pt-BR")
            translated_text = translate(text, 'pt', 'en')
            print(f"Você disse: {text}\n")
            return translated_text.lower()
        except sr.UnknownValueError:
            print("Não foi possível entender o áudio")
            return None
        except sr.RequestError as e:
            print(f"Erro na solicitação ao serviço de reconhecimento de voz: {e}")
            return None

In [6]:
# Função para traduzir texto
def translate(text, src_lang, dest_lang):
    translator = Translator()
    translated_text = translator.translate(text, src=src_lang, dest=dest_lang)
    return translated_text.text

In [7]:
# Função para reproduzir áudio
def play_audio(audio_text):
    myobj = gTTS(text=audio_text, lang="pt", slow=False)
    myobj.save("./audio/voice.mp3")
    mixer.init()
    mixer.music.load('./audio/voice.mp3')
    mixer.music.play()
    while mixer.music.get_busy():
        time.sleep(1)
    mixer.music.stop()
    mixer.music.unload()

## Prompts

### Pergunta escrita e resposta escrita

In [10]:
prompt = "Why is Minecraft amazing?"  # Mudar para o prompt desejado
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=50)
print("Generated response:", response)

Generated response: Who are you?

I'm a game designer, a programmer, and a creator. I love to build worlds and explore them, and I want to share that experience with as many people as possible.
One of the most incredible features of


### Resposta em áudio

In [20]:
prompt = "Why is Minecraft amazing?"  # Mudar para o prompt desejado
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=50)
translated_response = translate(response, 'en', 'pt')
print("Resposta:", translated_response)
play_audio(translated_response)

Resposta: Por que o Minecraft é incrível?É uma aventura sem fim em um mundo de infinitas possibilidades, onde sua imaginação é o único limite.Desde o momento em que você entra nesse universo virtual, você está envolvido por uma atmosfera única, onde todos os blocos e todas as criaturas escondem segredos e desafiam a espera para serem explorados.
Uma das características mais incríveis do Minecraft é sua natureza de sandbox.Aqui, você não é apenas um jogador, mas um criador.Com um movimento simples de mouse, você pode construir castelos majestosos, cidades movimentadas


### Prompt de fala + resposta em áudio

In [12]:
prompt = recognize_speech()  # Já traduz para o inglês
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=50)
translated_response = translate(response, 'en', 'pt')
print("Resposta:", translated_response)
play_audio(translated_response)

Ouvindo...

Você disse: Por que o Minecraft é mais do que apenas um jogo
Resposta: Por que o Minecraft é mais do que apenas um jogo.É uma aventura sem fim em um mundo de infinitas possibilidades, onde sua imaginação é o único limite.Desde o momento em que você entra nesse universo virtual, você está envolvido por uma atmosfera única, onde
